<a href="https://colab.research.google.com/github/Navya-prasad/YOLOv8n-based-Driver-Monitoring-System/blob/main/ECG_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
import os
import glob
from sklearn.model_selection import train_test_split
from pathlib import Path

# ==== PATHS ====
input_dir = "ecg_dataset_raw"       # folder with raw images
processed_dir = "ecg_dataset_processed"
train_dir = os.path.join(processed_dir, "train")
val_dir = os.path.join(processed_dir, "val")

# ==== PARAMETERS ====
crop_width = 2030
crop_height = 1180

# ==== CREATE FOLDERS ====
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

def preprocess_ecg(image_path):
    # Load image
    img = cv2.imread(image_path)

    # Step 1: Auto crop (centered crop to 2030x1180)
    h, w = img.shape[:2]
    start_x = (w - crop_width) // 2
    start_y = (h - crop_height) // 2
    cropped = img[start_y:start_y+crop_height, start_x:start_x+crop_width]

    # Step 2: Remove grid lines
    gray = cv2.cvtColor(cropped, cv2.COLOR_BGR2GRAY)

    # Otsu thresholding
    _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # Morphological opening to remove small lines
    kernel = np.ones((1, 1), np.uint8)
    morph = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)

    # Step 3: Canny Edge Detection
    edges = cv2.Canny(morph, 50, 150)

    # Step 4: Mask application (Inverse contour to keep ECG waveform)
    mask = cv2.bitwise_not(morph)
    result = cv2.bitwise_and(gray, gray, mask=mask)

    # Step 5: Histogram Equalization
    enhanced = cv2.equalizeHist(result)

    return enhanced

# ==== LOAD ALL IMAGES ====
image_paths = glob.glob(os.path.join(input_dir, "*.jpg")) + \
              glob.glob(os.path.join(input_dir, "*.png")) + \
              glob.glob(os.path.join(input_dir, "*.jpeg"))

# ==== TRAIN/VAL SPLIT ====
train_files, val_files = train_test_split(image_paths, test_size=0.2, random_state=42)

# ==== PROCESS & SAVE ====
for file_list, out_dir in [(train_files, train_dir), (val_files, val_dir)]:
    for img_path in file_list:
        processed_img = preprocess_ecg(img_path)
        filename = Path(img_path).name
        cv2.imwrite(os.path.join(out_dir, filename), processed_img)

print(f"Processing complete: {len(train_files)} train, {len(val_files)} val images saved.")
